In [1]:

!pip install sentence_transformers  ##installing hget in the requirements.txt file
!pip install torch.utils.data
!pip install datasets
!pip install transformers
import random
from transformers import MarianMTModel, MarianTokenizer
from typing import List
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 
import re
import pandas as pd
import random
from random import shuffle
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer 
from sentence_transformers import CrossEncoder 
from sentence_transformers import SentenceTransformer 
import pyarrow as pa
import os
import pyarrow.dataset as ds
from datasets import Dataset
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers.cross_encoder.evaluation import CECorrelationEvaluator 
from sentence_transformers.cross_encoder import CrossEncoder
from tqdm.auto import tqdm
from sentence_transformers import models, SentenceTransformer
from sentence_transformers.losses import CosineSimilarityLoss
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from numpy import mean
from numpy import std



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 4.1 MB/s 
     |████████████████████████████████| 4.7 MB 20.7 MB/s 
     |████████████████████████████████| 1.3 MB 42.4 MB/s 
     |████████████████████████████████| 120 kB 77.1 MB/s 
     |████████████████████████████████| 6.6 MB 67.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=eb61993f9a834078c6fdb1c4347640923f1bde9a2073a2aa5552895b9aa3d332
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch.utils.data (from versions: none)
ERROR: No matching distribution found for torch.utils.data
Looking in indexes: http

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [4]:
# import random
# import pandas as pd

class ADEA:
  def __init__(self):
    # self.input = input 
    self.PUNCTUATIONS = ['.', ',', '!', '?', ';', ':']
    # self.num_augs = [1]
    # self.PUNC_RATIO = 0.3
    

  # Insert punction words into a given sentence with the given ratio "punc_ratio"
  def insert_punctuation_marks(self, sentence, punc_ratio= 0.3):
    # punc_ratio = self.PUNC_RATIO
    random.seed(0)
    words = sentence.split(' ')
    new_line = []
    q = random.randint(1, int(punc_ratio * len(words) + 1))
    qs = random.sample(range(0, len(words)), q)
    for j, word in enumerate(words):
      if j in qs:
        new_line.append(self.PUNCTUATIONS[random.randint(0, len(self.PUNCTUATIONS)-1)])
        new_line.append(word)
      else:
        new_line.append(word)
    new_line = ' '.join(new_line)
    return new_line

  def main_sbert_(self, train_orig, num_augs=[1]):
    for aug in num_augs:
      data_aug = []      
      for line in train_orig:
        sentence = line
        for i in range(aug):
          sentence_aug = self.insert_punctuation_marks(sentence)
          data_aug.append(sentence_aug)
      df = pd.DataFrame({'translated_text': data_aug})
    return df['translated_text']

  def main(self, dataset, num_augs = []):
    dataset = '/content/train_test.txt'
    for aug in num_augs:
      data_aug = []
      with open(dataset , 'r') as train_orig:
        for line in train_orig:
          line1 = line.split('\t')
          label = line1[0]
          sentence = line1[1]
          for i in range(aug):
            sentence_aug = self.insert_punctuation_marks(sentence)
            line_aug = '\t'.join([label, sentence_aug])
            data_aug.append(line_aug)
          data_aug.append(line)

      with open('check_train_orig_plus_augs_' + str(aug) + '.txt', 'w') as train_orig_plus_augs:
        train_orig_plus_augs.writelines(data_aug)
      df = pd.DataFrame([sub.replace('\n','').split('\t') for sub in data_aug ])
      
      new_header = df.iloc[1]
      df = df[2:]
      df.columns = new_header
      df
    return df
      

In [5]:

# from transformers import MarianMTModel, MarianTokenizer


class BackTranslate:

  def __init__(self):
    # Get the name of the first model
    de_en_model_name = 'Helsinki-NLP/opus-mt-de-en'
    # Get the tokenizer
    de_en_model_tkn = MarianTokenizer.from_pretrained(de_en_model_name)
    # Load the pretrained model based on the name
    de_en_model = MarianMTModel.from_pretrained(de_en_model_name)

    ### Once data is translate from de to en use en-ro for step one of back translation
    # Get the name of the first model
    en_ro_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
    # Get the tokenizer
    en_ro_model_tkn = MarianTokenizer.from_pretrained(en_ro_model_name)
    # Load the pretrained model based on the name
    en_ro_model = MarianMTModel.from_pretrained(en_ro_model_name)


    ##this is for step 2 of back translation  backtranslation from Romance to en 
    # Get the name of the first model
    ro_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
    # Get the tokenizer
    ro_en_model_tkn = MarianTokenizer.from_pretrained(ro_en_model_name)
    # Load the pretrained model based on the name
    ro_en_model = MarianMTModel.from_pretrained(ro_en_model_name)



  def format_batch_texts(self,language_code, batch_texts):
    # batch_texts = self.batch_texts
    formated_bach = [">>{}<< {}".format(language_code, text) for text in batch_texts]
    return formated_bach

  def perform_translation(self, batch_texts, model, tokenizer, language="ROMANCE"):
      # Prepare the text data into appropriate format for the model
      formated_batch_texts = self.format_batch_texts(language, batch_texts)
      # Generate translation using model
      translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))
      # Convert the generated tokens indices back into text
      translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]      
      return translated_texts

  def back_translation(self,  get_text:list,model, tokenizer, language="en" ):
      # Prepare the text data into appropriate format for the model
      formated_batch_texts = self.format_batch_texts(language, get_text)
     # Generate translation using model
      translated = model.generate(**tokenizer(formated_batch_texts, return_tensors="pt", padding=True))
      # Convert the generated tokens indices back into text
      translated_texts = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
      return translated_texts


In [6]:
# from typing import List
# import pandas as pd

class DataCollect:
  def __init__(self,path):
    self.df = pd.read_csv(path, sep = '\t')
    self.path = path

  def datatypes(self):
    return self.df.dtypes

  def look_info(self):
    return self.df.iloc[1]

  def get_dataframe(self):
    return self.df

  def descriptive_stats(self):
    return self.df.describe(include='all')

  def get_columns(self):
    return self.df.columns
    
  def slice_dice(self, x):
    # x is len of data required
    self.df = self.df[0:x]
    self.df = self.df.fillna(' ')
    
  def required_cols(self,cols:List):    
    self.df['Text_'] = self.df[cols].apply(lambda x: ','.join(x.dropna().astype(str)),axis = 1)

  def return_data(self, label):
    self.df = self.df[[label, 'Text_']]
    # self.output = join(dirname(self.path), 'ori' + basename(self.path)) 
    # self.df.to_csv(self.output)
    return self.df



# df = pd.read_csv("/content/drive/MyDrive/MastersThesis/labeled_ai_data.tsv", sep = '\t')
# df = df[0:50]
# df = df.fillna(' ')
# df[['paragraph', 'keywords', 'description', 'title']]

# df['combined_text'] = df[['paragraph', 'keywords', 'description', 'title']].apply(lambda x: ','.join(x.dropna().astype(str)),axis = 1)

In [7]:
##EDA 

# import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# from nltk.corpus import wordnet 
# import re
# import pandas as pd
# import random
# from random import shuffle
from os.path import dirname, basename, join
random.seed(1)

class EDA:

  def __init__(self):
    #stop words list
    self.stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 
          'ours', 'ourselves', 'you', 'your', 'yours', 
          'yourself', 'yourselves', 'he', 'him', 'his', 
          'himself', 'she', 'her', 'hers', 'herself', 
          'it', 'its', 'itself', 'they', 'them', 'their', 
          'theirs', 'themselves', 'what', 'which', 'who', 
          'whom', 'this', 'that', 'these', 'those', 'am', 
          'is', 'are', 'was', 'were', 'be', 'been', 'being', 
          'have', 'has', 'had', 'having', 'do', 'does', 'did',
          'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or',
          'because', 'as', 'until', 'while', 'of', 'at', 
          'by', 'for', 'with', 'about', 'against', 'between',
          'into', 'through', 'during', 'before', 'after', 
          'above', 'below', 'to', 'from', 'up', 'down', 'in',
          'out', 'on', 'off', 'over', 'under', 'again', 
          'further', 'then', 'once', 'here', 'there', 'when', 
          'where', 'why', 'how', 'all', 'any', 'both', 'each', 
          'few', 'more', 'most', 'other', 'some', 'such', 'no', 
          'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 
          'very', 's', 't', 'can', 'will', 'just', 'don', 
          'should', 'now', '']

#cleaning up text

  def get_only_chars(self,line):
      clean_line = ""
      line = line.replace("’", "")
      line = line.replace("'", "")
      line = line.replace("-", " ") #replace hyphens with spaces
      line = line.replace("\t", " ")
      line = line.replace("\n", " ")
      line = line.lower()

      for char in line:
          if char in 'qwertyuiopasdfghjklzxcvbnm ':
              clean_line += char
          else:
              clean_line += ' '

      clean_line = re.sub(' +',' ',clean_line) #delete extra spaces
      if clean_line[0] == ' ':
          clean_line = clean_line[1:]
      return clean_line

  ########################################################################
  # Synonym replacement
  # Replace n words in the sentence with synonyms from wordnet
  ########################################################################

  #for the first time you use wordnet


  def synonym_replacement(self,words, n):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in self.stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
      synonyms = self.get_synonyms(random_word)
      if len(synonyms) >= 1:
        synonym = random.choice(list(synonyms))
        new_words = [synonym if word == random_word else word for word in new_words]
        #print("replaced", random_word, "with", synonym)
        num_replaced += 1
      if num_replaced >= n: #only replace up to n words
        break

    #this is stupid but we need it, trust me
    sentence = ' '.join(new_words)
    new_words = sentence.split(' ')

    return new_words

  def get_synonyms(self,word):
    synonyms = set()
    for syn in wordnet.synsets(word): 
      for l in syn.lemmas(): 
        synonym = l.name().replace("_", " ").replace("-", " ").lower()
        synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
        synonyms.add(synonym) 
    if word in synonyms:
      synonyms.remove(word)
    return list(synonyms)

########################################################################
# Random deletion
# Randomly delete words from the sentence with probability p
########################################################################

  def random_deletion(self,words, p):

    #obviously, if there's only one word, don't delete it
    if len(words) == 1:
      return words

    #randomly delete words with probability p
    new_words = []
    for word in words:
      r = random.uniform(0, 1)
      if r > p:
        new_words.append(word)

    #if you end up deleting all words, just return a random word
    if len(new_words) == 0:
      rand_int = random.randint(0, len(words)-1)
      return [words[rand_int]]

    return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################

  def random_swap(self,words, n):
    new_words = words.copy()
    for _ in range(n):
      new_words = self.swap_word(new_words)
    return new_words

  def swap_word(self,new_words):
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    while random_idx_2 == random_idx_1:
      random_idx_2 = random.randint(0, len(new_words)-1)
      counter += 1
      if counter > 3:
        return new_words
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################

  def random_insertion(self,words, n):
    new_words = words.copy()
    for _ in range(n):
      self.add_word(new_words)
    return new_words

  def add_word(self,new_words):
    synonyms = []
    counter = 0
    while len(synonyms) < 1:
      random_word = new_words[random.randint(0, len(new_words)-1)]
      synonyms = self.get_synonyms(random_word)
      counter += 1
      if counter >= 10:
        return
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

########################################################################
# main data augmentation function
########################################################################

  def eda(self,sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

    
    sentence = self.get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)
    
    
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1

    #sr
    if (alpha_sr > 0):
      n_sr = max(1, int(alpha_sr*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    #ri
    if (alpha_ri > 0):
      n_ri = max(1, int(alpha_ri*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    #rs
    if (alpha_rs > 0):
      n_rs = max(1, int(alpha_rs*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    #rd
    if (p_rd > 0):
      for _ in range(num_new_per_technique):
        a_words = self.random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [self.get_only_chars(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
      augmented_sentences = augmented_sentences[:num_aug]
    else:
      keep_prob = num_aug / len(augmented_sentences)
      augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    
   
    augmented_sentences.append(sentence)

    return augmented_sentences 

#### to be changed and merged above
  def eda_aug(self,sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):

    
    sentence = self.get_only_chars(sentence)
    words = sentence.split(' ')
    words = [word for word in words if word is not '']
    num_words = len(words)
    
    
    augmented_sentences = []
    num_new_per_technique = int(num_aug/4)+1

    #sr
    if (alpha_sr > 0):
      n_sr = max(1, int(alpha_sr*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.synonym_replacement(words, n_sr)
        augmented_sentences.append(' '.join(a_words))

    #ri
    if (alpha_ri > 0):
      n_ri = max(1, int(alpha_ri*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.random_insertion(words, n_ri)
        augmented_sentences.append(' '.join(a_words))

    #rs
    if (alpha_rs > 0):
      n_rs = max(1, int(alpha_rs*num_words))
      for _ in range(num_new_per_technique):
        a_words = self.random_swap(words, n_rs)
        augmented_sentences.append(' '.join(a_words))

    #rd
    if (p_rd > 0):
      for _ in range(num_new_per_technique):
        a_words = self.random_deletion(words, p_rd)
        augmented_sentences.append(' '.join(a_words))

    augmented_sentences = [self.get_only_chars(sentence) for sentence in augmented_sentences]
    shuffle(augmented_sentences)

    #trim so that we have the desired number of augmented sentences
    if num_aug >= 1:
      augmented_sentences = augmented_sentences[:num_aug]
    else:
      keep_prob = num_aug / len(augmented_sentences)
      augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]

    #append the original sentence
    


    return augmented_sentences 



  def gen_eda(self,train_orig, alpha_sr = 0.1, alpha_ri = 0.1, alpha_rs= 0.1, alpha_rd= 0.1, num_aug=9):

      train_orig = '/content/train_test.txt'
      output_file = join(dirname(train_orig), 'eda_' + basename(train_orig))

      writer = open(output_file, 'w')
      lines = open(train_orig, 'r').readlines()
      aug = []
      for i, line in enumerate(lines):

          parts = line[:-1].split('\t')
          label = parts[0]
          sentence = parts[1]
          aug_sentences  = self.eda(sentence, alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
          # print(aug_sentences)
          for i in range(len(aug_sentences)):
            line_aug = '\t'.join([label, aug_sentences[i]])
            aug.append(line_aug)


          
           

          for aug_sentence in aug_sentences:
              writer.write(label + "\t" + aug_sentence + '\n')



      writer.close()

      df = pd.DataFrame([sub.replace('\n','').split('\t') for sub in aug ], index= None)
      len_ = num_aug + 1
      new_header = df.iloc[num_aug]
      df = df[len_:]
      df.columns = new_header
      df
      print("generated augmented sentences with eda for " + train_orig + " to " + output_file + " with num_aug=" + str(num_aug))
      return df



  def gen_eda_for_sbert_(self, df , alpha_sr = 0.1, alpha_ri = 0.1 , alpha_rs = 0.1, alpha_rd = 0.1, num_aug=1): 

      aug_data = []
      for sentence in df:
        
        just_augs  = self.eda_aug(sentence, alpha_sr=alpha_sr, alpha_ri=alpha_ri, alpha_rs=alpha_rs, p_rd=alpha_rd, num_aug=num_aug)
        # print(type(just_augs))
        aug_data.append(just_augs[0])
        

      df = pd.DataFrame({'translated_text': aug_data})
        
      return df['translated_text']

In [44]:


class Sbert:

  def __init__(self , df_sbert):
    # Get the name of the first model
    self.de_en_model_name = 'Helsinki-NLP/opus-mt-de-en'
    # Get the tokenizer
    self.de_en_model_tkn = MarianTokenizer.from_pretrained(self.de_en_model_name)
    # Load the pretrained model based on the name
    self.de_en_model = MarianMTModel.from_pretrained(self.de_en_model_name)

    ### Once data is translate from de to en use en-ro for step one of back translation
    # Get the name of the first model
    self.en_ro_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
    # Get the tokenizer
    self.en_ro_model_tkn = MarianTokenizer.from_pretrained(self.en_ro_model_name)
    # Load the pretrained model based on the name
    self.en_ro_model = MarianMTModel.from_pretrained(self.en_ro_model_name)

    ##this is for step 2 of back translation  backtranslation from Romance to en 
    # Get the name of the first model
    self.ro_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
    # Get the tokenizer
    self.ro_en_model_tkn = MarianTokenizer.from_pretrained(self.ro_en_model_name)
    # Load the pretrained model based on the name
    self.ro_en_model = MarianMTModel.from_pretrained(self.ro_en_model_name)
    self.df_sbert = df_sbert
    self.adea = ADEA()
    self.eda = EDA()
    

    
    


  def get_dataframe(self):
    return self.df_sbert



  ##### other important definations 
  ###TF_IDF

  def len_vocab(self,df1,df2):


    self.vocab_a = set()

    for i in range(0,len(df1)):
      self.vocab_a.update(set(df1.iloc[i].split()))


    self.vocab_b = set()

    for i in range(0,len(df2)):
      self.vocab_b.update(set(df2.iloc[i].split()))
    return self.vocab_a, self.vocab_b


  
  def tfidf(self,word,sentence,x):
    
    #term frequecny 
    tf = sentence.count(word)/len(sentence) 
    #inverse trans 
    try:
      idf = np.log(len(x)/sum([1 for doc in x if word in doc]) )

    except ZeroDivisionError:
      idf = 0
    return round(tf*idf, 4)

  def find_vec(self,x,len_vocab,df):
    vec_a = []

    # for word in vocab_a:
    for word in len_vocab:
      # vec_a.append(tfidf(word,x.split(),df_new['de-english']))
      vec_a.append(self.tfidf(word,x.split(),df))
    return vec_a

  def get_embedding_vector_df(self,df1,df2, func):

    self.vocab_a , self.vocab_b = self.len_vocab(df1,df2)

    # df_tf_idf = pd.DataFrame(
    if len(self.vocab_a) > len(self.vocab_b):

      df = pd.DataFrame(
          {
              'sentence1': df1.apply(lambda x: func(x,self.vocab_a,df1)),
              'sentence2': df2.apply(lambda x: func(x,self.vocab_b,df2))
          }
      )
    elif len(self.vocab_a) > len(self.vocab_b):

      df = pd.DataFrame(
          {
              'sentence1': df1.apply(lambda x: func(x,self.vocab_a,df1)),
              'sentence2': df2.apply(lambda x: func(x,self.vocab_b,df2))
          }
      ) 

    else:


      df = pd.DataFrame(
          {
              'sentence1': df1.apply(lambda x: func(x,self.vocab_a,df1)),
              'sentence2': df2.apply(lambda x: func(x,self.vocab_b,df2))
          }
      )


    return df

     
  def check_cosine(self,df1, df2):
    simi = []

    for i in range(0,len(df1)):
      simi.append(cosine_similarity([np.array(df1[i])],[np.array(df2[i])])[0][0])

    return simi

  ###TF-IDF from SKlearn
  


  def get_tf_idf_sklearn(self,df1,df2):
    vectorizer = TfidfVectorizer(lowercase= True)

    X_train_tf = pd.concat([df1, df2]).unique()
    vectorizer.fit(X_train_tf)
    #embeddings 

    sent1 = vectorizer.transform(df1)
    sent2 = vectorizer.transform(df2)

    return np.diag(cosine_similarity(sent1, sent2))

  ###CE
  #sbert_cross_encoder


  def check_score_ce(self,df1,df2):
    model_ce_trans = CrossEncoder('cross-encoder/stsb-roberta-base') 

    sentence_pairing = []

    for sent1, sent2 in zip(df1,df2):
      sentence_pairing.append([sent1,sent2])

    score = model_ce_trans.predict(sentence_pairing)

    return score

  ###Bi-enocder
  


  def check_score_bi(self,df1,df2):
    model_bi_trans = SentenceTransformer('stsb-mpnet-base-v2')

    senet1_emb = model_bi_trans.encode(df1, show_progress_bar =True)
    senet2_emb = model_bi_trans.encode(df2, show_progress_bar =True) 

    score = self.check_cosine(senet1_emb, senet2_emb)

    return score


  ###BM25
  
  def bm25(self,word,sentence,x, k =1.2, b =0.75):
    ##avg len of all the sentences
    avgdl = sum((len(sentence) for sentence in [a.split() for a in x ]))/len(x)
    N = len(x) 
    
    #term frequecny 
    freq = sentence.count(word)
    try:
      tf = (freq * (k + 1))/(freq + k * (1- b + b * len(sentence)/avgdl))
    except ZeroDivisionError:
      tf = 0


    #inverse trans 
    N_q = sum([1 for doc in x if word in doc])
    try:
      idf = np.log(((N - N_q + 0.5)/(N_q + 0.5)) +1)
    except ZeroDivisionError:
      idf = 0
    return round(tf*idf, 4)

  def find_vec_bm25(self,x,len_vocab,df):
    vec_a = []

    for word in len_vocab:
      vec_a.append(self.bm25(word,x.split(),df))
    return vec_a


  ###Others if possible

  #####
  ################################################
  ###### Methods for generating second columns
  ################################################

  def back_translate_sbert(self,data):
    #data is column in the dataframe which has to be translated
    self.back_translate = BackTranslate()
    tt = self.back_translate
    to_ro_translate = tt.perform_translation(data, self.en_ro_model, self.en_ro_model_tkn)
    to_en_translate = tt.back_translation(to_ro_translate, self.ro_en_model,self.ro_en_model_tkn)

    self.df_sbert['translated_text'] = to_en_translate
    return  self.df_sbert

     

  def aeda_translate(self,data):
    adea = self.adea
    text = adea.main_sbert_(data,[1])
    self.df_sbert['translated_text'] = text
    return self.df_sbert

  def eda_translate(self,data):
    EDA_ = self.eda
    text =  EDA_.gen_eda_for_sbert_(data, 0.1,0.1,0.1,0.1)
    self.df_sbert['translated_text'] = text
    return self.df_sbert

  ################################################
  ###create data aug def
  ################################################

  def choose_aug_stratergy(self,df1):
    self.aug_choose = input("Select the method of augmentation:")

    if self.aug_choose == "backtranslation":
      self.back_translate_sbert(df1) 
    elif self.aug_choose == "EDA":
      self.eda_translate(df1)
    elif self.aug_choose == "AEDA":
      self.aeda_translate(df1)
    else:
      print('please choose one of 3 stratergies')
    return self.df_sbert




  #################################################
  ##Similarity check
  #################################################

  def similarity_check(self,df1,df2):
    ## df1 and df2 are 2 coulmns which we are about to compare
    ## df1 = df_sbert['text']
    ## df2 = df_sbert['translated_text']

    ##get tf-idf
    # vocab_a,vocab_b = len_vocab(df1, df2)
    # vocab_a,vocab_b = self.vocab_a , self.vocab_b

    self.meth = input("Select the method of similarity:")

    if self.meth == 'tf-idf':

      
      # tf_embedding_df = self.get_embedding_vector_df(df1, df2, self.vocab_a, self.vocab_b,self.find_vec)
      tf_embedding_df = self.get_embedding_vector_df(df1, df2,self.find_vec)
      self.df_sbert[f'{self.meth}_score'] = self.check_cosine(tf_embedding_df['sentence1'],tf_embedding_df['sentence2'])

    elif self.meth == 'ce': 

      self.df_sbert[f'{self.meth}_score'] = self.check_score_ce(df1, df2)

    elif self.meth == 'bm25': 

      bm_embedding_df = self.get_embedding_vector_df(df1, df2,self.find_vec_bm25)

      self.df_sbert[f'{self.meth}_score'] = self.check_cosine(bm_embedding_df['sentence1'],bm_embedding_df['sentence2'])

    elif self.meth == 'BI': 

      self.df_sbert[f'{self.meth}_score'] = self.check_score_bi(df1, df2)
    else:
      print('Please select one of 4 strategies')

    return self.df_sbert
  
  def gen_train_dev_ce(self,df,x):
    ##df is dataframe
    ##x is how big you want to train dataset

    # train_data = []
    hg_df_new = Dataset(pa.Table.from_pandas(df[0:x]))
    hg_df_new_dev = Dataset(pa.Table.from_pandas(df[x:len(df)]))

    return hg_df_new,hg_df_new_dev

  def finetuning_ce(self,df_train, df_dev):
    ##fine tuning is for genrating loader and evalutor
    ##df_train is trains set of the data
    ##df_dev is  dev set of data used for the evaluator

    batch_size = 2
    ##loading our training data in the dataloader 
    train_data = []

    for row in df_train:
      train_data.append( 
          InputExample(
              ## need to change
              texts = [row['text'], row['translated_text']],
              label = float(row[f'{self.meth}_score'])
          )
      )
    loader = DataLoader(train_data, shuffle = True, batch_size=batch_size) 

    # return loader 

        ##Evaluator
    

    dev_data = []

    for row in df_dev:
      dev_data.append(
          InputExample(
              ## need to change
              texts = [row['text'], row['translated_text']],
              label = float(row[f'{self.meth}_score'])
          )
      )


    #the dev data goes into the evaluator 

    evaluator = CECorrelationEvaluator.from_input_examples(dev_data)

    # return evaluator 

    # label2int = {"1": 1, "0": 0}
    self.finetuned_ce = CrossEncoder('bert-base-uncased', num_labels=1)

    no_epoch_num = 1 
    warmup = int(len(loader)*0.4*no_epoch_num)

    self.finetuned_ce.fit(
        train_dataloader = loader,
        evaluator = evaluator,
        epochs = no_epoch_num,
        warmup_steps=warmup,
        output_path = 'bert-istari-ce'
        # optimizer_params={'lr': 5e-6}
    )

    ## output path i can take seperately 

    return self.finetuned_ce


    
  
  def generate_silverdata(self,train):
    ## first generate gold dataset
    ## than use finetuned_ce to get 
   

    gold = pd.DataFrame({
        'sentence1': train['text'],
        'sentence2': train['translated_text'],
        'label': train[f'{self.meth}_score'] #not necessary,
        # 'ori' : train['label']
    })

    

    pairs = pd.DataFrame({
        'sentence1': [],
        'sentence2': []
        # 'ori':[]
    })

    for sentence1 in tqdm(list(set(gold['sentence1']))):
      sampled = gold[gold['sentence1'] != sentence1].sample(2)
      sampled = sampled['sentence2'].tolist() 
      for sentence2 in sampled:
        pairs = pairs.append({
            'sentence1': sentence1,
            'sentence2': sentence2

        }, ignore_index = True)


    ##insert the model which has been trained earlier
    # ce = CrossEncoder('bert-istari-ce')

## use this cross encoder to create lables for silver datset 


    silver = list(zip(pairs['sentence1'], pairs['sentence2']) )
    # scores = ce.predict(silver) 
    scores = self.finetuned_ce.predict(silver)

    pairs['label'] = scores

    self.df_all_data = gold.append(pairs, ignore_index= True)

    return self.df_all_data
  
  def finetuning_bi_encoder(self,all_df_data):
    batch_size = 2 
    #all_df_data is combination of gold +silver  
    train_data_ = []
    for i, row in self.df_all_data.iterrows():
      train_data_.append(
          InputExample(
              texts=[row['sentence1'],row['sentence2']],
              label =float(row['label'])
          )
      )

    
    loader_ = DataLoader(
        train_data_, shuffle=True, batch_size=batch_size
    )


        ## can chnage 'bert-base-uncased'
    bert = models.Transformer('bert-base-uncased')
    pooling = models.Pooling(bert.get_word_embedding_dimension(),
                            pooling_mode_mean_tokens=True)
    
    model_bi_enc = SentenceTransformer(modules=[bert, pooling])

     
    loss = CosineSimilarityLoss(model = model_bi_enc)

    num_epochs = 1 

    warmup= int(len(loader_)*0.10*num_epochs)

    model_bi_enc.fit(
        train_objectives= [(loader_,loss)],
        epochs=num_epochs,
        warmup_steps=warmup,
        output_path='bert-istari-aug'


    )



     


    return model_bi_enc


In [9]:




from google.colab import drive
drive.mount('/content/drive')

class AllStrategy:
    def __init__(self,path):
                # Get the name of the first model
        self.de_en_model_name = 'Helsinki-NLP/opus-mt-de-en'
        # Get the tokenizer
        self.de_en_model_tkn = MarianTokenizer.from_pretrained(self.de_en_model_name)
        # Load the pretrained model based on the name
        self.de_en_model = MarianMTModel.from_pretrained(self.de_en_model_name)

        ### Once data is translate from de to en use en-ro for step one of back translation
        # Get the name of the first model
        self.en_ro_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
        # Get the tokenizer
        self.en_ro_model_tkn = MarianTokenizer.from_pretrained(self.en_ro_model_name)
        # Load the pretrained model based on the name
        self.en_ro_model = MarianMTModel.from_pretrained(self.en_ro_model_name)

        ##this is for step 2 of back translation  backtranslation from Romance to en 
        # Get the name of the first model
        self.ro_en_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
        # Get the tokenizer
        self.ro_en_model_tkn = MarianTokenizer.from_pretrained(self.ro_en_model_name)
        # Load the pretrained model based on the name
        self.ro_en_model = MarianMTModel.from_pretrained(self.ro_en_model_name)
        self.dc = DataCollect(path)
        self.eda = EDA()
        self.aeda = ADEA()
        self.back_translate = BackTranslate()
        pass

    def prep_data(self, len):
        #len length of data or records to be taken in consideration should be more than 10
        
        self.dc.slice_dice(len)
        self.dc.required_cols(['paragraph', 'keywords', 'description', 'title'])
        df = self.dc.return_data('labels')
        tt = self.back_translate
        en_translate = tt.back_translation(df['Text_'],self.de_en_model,self.de_en_model_tkn)
        df_new = pd.DataFrame({'label': pd.get_dummies(df['labels'])['Information'] ,'text': en_translate })
        df_new.to_csv( 'train_test.txt' , sep = '\t' , index= False) 
        return df_new


    

    def stratergy1_aug_sbert(self,df_new):
    ###### new func
        
        self.sbert = Sbert(df_new)
        val = round(0.7 * len(df_new))

        sbert = self.sbert
        df_sbert = sbert.get_dataframe()
        # df_sbert

        ##choose either EDA, AEDA or backtranslate
        df_sbert = sbert.choose_aug_stratergy(df_sbert['text'])
        # df_sbert  
        ##choose one of 4 similarity check stratergies i.e ce, tf-idf, bm25 and bi
        df_sbert = sbert.similarity_check(df_sbert['text'], df_sbert['translated_text'])

        # df_sbert 
        ## have to change 3 to 70:30 for the dataset
        train , dev = sbert.gen_train_dev_ce(df_sbert, val)

        ce_model = sbert.finetuning_ce(train,dev)

        all_data = sbert.generate_silverdata(train) 

        bi_model = sbert.finetuning_bi_encoder(all_data)

        return bi_model

    def strategy2_AEDA(self,path_eda, num = []):
        
        df = self.aeda.main(path_eda, num_augs= num)
        
        return df



    path_eda = '/content/train_test.txt'
    path_eda

    def strategy3_EDA(self,path_eda, num):

        
        df = self.eda.gen_eda(path_eda, 0.1, 0.1, 0.1, 0.1 , num_aug= num)
        return df











Mounted at /content/drive


In [29]:
import os
from sentence_transformers import SentenceTransformer 
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import classification_report 
from sklearn.metrics import matthews_corrcoef 
import pandas as pd
from imblearn.over_sampling import SMOTE 
import os
import nltk
import nltk.corpus
from nltk.probability import FreqDist
from matplotlib.backends.backend_pdf import PdfPages
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')



####################
##Text mining
###################

##get the text column from df
##convert it to list

def text_mine(df_new):
    #df_new is the txt col
    df_new['text'].tolist()
    all_tokens = []

    for i in df_new['text'].tolist():
        tokens = nltk.word_tokenize(i)
        for j in tokens:
            all_tokens.append(j)

    CHARS_TO_REMOVE = '''!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'''
    all_tokens = [x for x in all_tokens if x not in CHARS_TO_REMOVE]

    nltk.download('stopwords')
    stopwords = nltk.corpus.stopwords.words('english')
    all_tokens = [x for x in all_tokens if x not in stopwords]

    ## Most common tokens

    

    fdist = FreqDist()
    for word in all_tokens:
        fdist[word.lower()]+=1

    wholedict = pd.DataFrame({'words': fdist.keys(), 'count': fdist.values()})
    wholedict.sort_values(by=['count'] , ascending= False, inplace = True)

    top_10 = wholedict[0:10].plot( x ='words' , kind = 'bar' )
    pp = PdfPages("top_10_words.pdf")
    pp.savefig(top_10, bbox_inches='tight')
    pp.close()

    ######Distribution by type of word
    # all_tokens
    
    just_nouns = nltk.pos_tag(all_tokens)
    tag_fd = nltk.FreqDist(tag for (word, tag) in just_nouns)

    type_df = pd.DataFrame(tag_fd.most_common())
    type_df.columns = ['type', 'count']
    type_df.plot( x ='type' , kind = 'bar' )
### get fdist as resukt
###################
###################


#######Base Model##################

def basemodel(df):
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = model.encode(df['text'])
    # print(embeddings)
    return embeddings


def eda(data):

    path_eda = os.getcwd()+'\\train_test.txt'

    aug_df = data.strategy3_EDA(path_eda, 1)
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = model.encode(aug_df['text'].to_list())
    return embeddings


#check for the proper solution
def aeda(data):
    path_eda = os.getcwd()+'\\train_test.txt'
    aug_df = data.strategy2_AEDA(path_eda, [1])
    
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    embeddings = model.encode(aug_df['text'].to_list())
    # print(embeddings)
    return embeddings

def aug_sbert(df, data):
        
    aug_df_model = data.stratergy1_aug_sbert(df)
    # model = SentenceTransformer(aug_df)
    ## not using sentence transformer model we are using finetuned model which is mix of CE and BI
    embeddings = aug_df_model.encode(df['text'])
    # print(embeddings)
    return embeddings






#############################
#ML models 
#############################
def train_test(embeddings, target):
    X_train, X_test, y_train, y_test = train_test_split(embeddings, target, test_size= 0.33, random_state= 42)
    return X_train, X_test, y_train, y_test

def evaluate_model(model, X, y):

	cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
	return scores

def get_voting():
    # define the base models
    models = list()
    models.append(('LR', LogisticRegression()))
    models.append(('svm', SVC()))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():

    model_ = dict()
    model_['logistic'] = LogisticRegression()
    model_['cart'] = DecisionTreeClassifier()
    model_['svm'] = SVC()

    model_['hard_voting'] = get_voting()
    return model_

def ml_model(X_train,y_train, model:str):
    
    if model == 'LR':
        model_ = LogisticRegression(random_state=0,solver='lbfgs', class_weight='balanced').fit(X_train, y_train)
    elif model == 'SVM':
        model_ = SVC().fit(X_train,y_train) 
    elif model == 'Ensemble':
        estimators = []
        model1 = LogisticRegression()
        estimators.append(('logistic', model1))
        model2 = DecisionTreeClassifier()
        estimators.append(('cart', model2))
        model3 = SVC()
        estimators.append(('svm', model3))

        model_ = VotingClassifier(estimators, voting = 'hard')
        model_ = model_.fit(X_train, y_train)
        kfold = model_selection.KFold(n_splits= 2)
        results = model_selection.cross_val_score(model_, X_train, y_train, cv= kfold)
        print('results:', results)
        print('Mean results', results.mean())

    else:
        print('select one of the models')
    return model_ 



#output of ml_model to be used in predict
def predict(model_, X_test):
    pred= model_.predict(X_test)
    return pred 

#use output of predict func and y_test from train_test func in below func
def class_fication(actual, predict):
  target_names = ['0','1']
  report = classification_report(actual, predict, target_names = target_names, zero_division= True)
  matt_report = matthews_corrcoef(actual, predict)
  # print(report)
  return report, matt_report




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#Experiment performed  for 50 records

In [32]:

##########################Steps to be done
#################################
#################################
# checking on 50 records
#################################
#################################



path = "/content/drive/MyDrive/MastersThesis/labeled_ai_data.tsv"
data = AllStrategy(path) 
df = data.prep_data(50)

target = df['label']

# ###################################
# #### undersampling

# if target.value_counts()[1] > target.value_counts()[0] :
#     df1 = df[df['label'] == 1][0: target.value_counts()[0]]
#     df2 = df[df['label'] == 0]
#     df_new = df1.append(df2)
# else:
#     df1 = df[df['label'] == 0][0: target.value_counts()[1]]
#     df2 = df[df['label'] == 1]
#     df_new = df1.append(df2)

# df_new








/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [23]:
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

#With Logistic Regression

#Basemodel

In [33]:
####Embedings by baseline 

def run_with_base_model(df,ml_type:str):
  embedding = basemodel(df)


  X_train, X_test, y_train, y_test = train_test(embedding, target) 

  clf = ml_model(X_train,y_train, ml_type)

  pred = clf.predict(X_test) 

  confmat, matt = class_fication(y_test, pred)
  print ("Accuracy :", np.round(accuracy_score(y_test, pred),5))
  print ("Precision :", np.round(precision_score(y_test, pred),5))
  print ("Recall :", np.round(recall_score(y_test, pred),5))
  print ("f1 score :", np.round(f1_score(y_test, pred),5))
  print ("AUC score :", np.round(roc_auc_score(y_test, pred),5))

  print("Matrix",confmat)
  print("matthews Correlation coeffiecient:",matt) 
  if ml_type == "Ensemble":
    # evaluate the models and store results
    results, names = list(), list()
    models = get_models()
    for name, model in models.items():
      scores = evaluate_model(model, X_train, y_train)
      results.append(scores)
      names.append(name)
      print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [35]:
run_with_base_model(df,"LR")

Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0


#DA mit EDA 
### with number of augmentation 1 , 2, 3

In [37]:


####Embedings by eda 
def run_with_eda(data, ml_type):
  embedding = eda(data)
 
  eda_df = pd.read_csv('eda_train_test.txt', sep = '\t', header = 1)

  target_eda = eda_df['label']
  X_train, X_test, y_train, y_test = train_test(embedding, target_eda) 

  clf = ml_model(X_train,y_train, ml_type)

  pred = clf.predict(X_test) 

  confmat, matt = class_fication(y_test, pred)

  print ("Accuracy :", np.round(accuracy_score(y_test, pred),5))
  print ("Precision :", np.round(precision_score(y_test, pred),5))
  print ("Recall :", np.round(recall_score(y_test, pred),5))
  print ("f1 score :", np.round(f1_score(y_test, pred),5))
  print ("AUC score :", np.round(roc_auc_score(y_test, pred),5))

  print("Matrix",confmat)
  print("matthews Correlation coeffiecient:",matt) 
  if ml_type == "Ensemble":
  # evaluate the models and store results
    results, names = list(), list()
    models = get_models()
    for name, model in models.items():
      scores = evaluate_model(model, X_train, y_train)
      results.append(scores)
      names.append(name)
      print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

In [38]:
run_with_eda(data, "LR")

generated augmented sentences with eda for /content/train_test.txt to /content/eda_train_test.txt with num_aug=1
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
f1 score : 1.0
AUC score : 1.0
Matrix               precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

matthews Correlation coeffiecient: 1.0


#DA With Aeda

In [40]:


####Embedings by aeda 
def run_with_adea(data, ml_type):
  embedding = aeda(data)
  embedding.shape 
  aeda_df = pd.read_csv('check_train_orig_plus_augs_1.txt', sep = '\t', header = 1)
  target_aeda = aeda_df['label']
  X_train, X_test, y_train, y_test = train_test(embedding, target_aeda) 

  clf = ml_model(X_train,y_train, ml_type)

  pred = clf.predict(X_test) 

  confmat, matt = class_fication(y_test, pred)
  print ("Accuracy :", np.round(accuracy_score(y_test, pred),5))
  print ("Precision :", np.round(precision_score(y_test, pred),5))
  print ("Recall :", np.round(recall_score(y_test, pred),5))
  print ("f1 score :", np.round(f1_score(y_test, pred),5))
  print ("AUC score :", np.round(roc_auc_score(y_test, pred),5))

  print("Matrix",confmat)
  print("matthews Correlation coeffiecient:",matt) 
  if ml_type == "Ensemble":
  # evaluate the models and store results
    results, names = list(), list()
    models = get_models()
    for name, model in models.items():
      scores = evaluate_model(model, X_train, y_train)
      results.append(scores)
      names.append(name)
      print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))


In [41]:
run_with_adea(data, "LR")

Accuracy : 1.0
Precision : 1.0
Recall : 1.0
f1 score : 1.0
AUC score : 1.0
Matrix               precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

matthews Correlation coeffiecient: 1.0


# AugSBERT

In [42]:

###Embedings by aug_sbert

def run_with_augsbert(df,data,ml_type):
  embedding = aug_sbert(df, data)

  X_train, X_test, y_train, y_test = train_test(embedding, target) 

  clf = ml_model(X_train,y_train, ml_type)

  pred = clf.predict(X_test) 

  confmat, matt = class_fication(y_test, pred)

  print ("Accuracy :", np.round(accuracy_score(y_test, pred),5))
  print ("Precision :", np.round(precision_score(y_test, pred),5))
  print ("Recall :", np.round(recall_score(y_test, pred),5))
  print ("f1 score :", np.round(f1_score(y_test, pred),5))
  print ("AUC score :", np.round(roc_auc_score(y_test, pred),5))

  print("Matrix",confmat)
  print("matthews Correlation coeffiecient:",matt)
  if ml_type == "Ensemble":
  # evaluate the models and store results
    results, names = list(), list()
    models = get_models()
    for name, model in models.items():
      scores = evaluate_model(model, X_train, y_train)
      results.append(scores)
      names.append(name)
      print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores))) 


#With AugSBERT + EDA

###DA = EDA 
###similarity = ce

In [43]:
run_with_augsbert(df,data,"LR")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:EDA
Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0


###AugSBERT mit backtransation
###DA = backtranslation
###similarity = ce

In [45]:
run_with_augsbert(df,data,"LR")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:backtranslation


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy : 0.6
Precision : 0.66667
Recall : 0.66667
f1 score : 0.66667
AUC score : 0.58333
Matrix               precision    recall  f1-score   support

           0       0.50      0.50      0.50         2
           1       0.67      0.67      0.67         3

    accuracy                           0.60         5
   macro avg       0.58      0.58      0.58         5
weighted avg       0.60      0.60      0.60         5

matthews Correlation coeffiecient: 0.16666666666666666


#AugSBERT mit aeda

###DA = AEDA
###similarity = ce

In [46]:
run_with_augsbert(df,data,"LR")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:AEDA
Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

Accuracy : 0.4
Precision : 0.5
Recall : 0.66667
f1 score : 0.57143
AUC score : 0.33333
Matrix               precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.50      0.67      0.57         3

    accuracy                           0.40         5
   macro avg       0.25      0.33      0.29         5
weighted avg       0.30      0.40      0.34         5

matthews Correlation coeffiecient: -0.4082482904638631


#With Ensemble models

##Basemodel

In [47]:
####Embedings by baseline 
run_with_base_model(df,"Ensemble")


results: [0.8 0.6]
Mean results 0.7
Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0
>logistic 0.694 (0.039)
>cart 0.537 (0.176)
>svm 0.694 (0.039)
>hard_voting 0.694 (0.039)


##EDA

In [48]:
##By EDA
run_with_eda(data, "Ensemble")

generated augmented sentences with eda for /content/train_test.txt to /content/eda_train_test.txt with num_aug=1
results: [0.6 0.7]
Mean results 0.6499999999999999
Accuracy : 0.9
Precision : 0.875
Recall : 1.0
f1 score : 0.93333
AUC score : 0.83333
Matrix               precision    recall  f1-score   support

           0       1.00      0.67      0.80         3
           1       0.88      1.00      0.93         7

    accuracy                           0.90        10
   macro avg       0.94      0.83      0.87        10
weighted avg       0.91      0.90      0.89        10

matthews Correlation coeffiecient: 0.7637626158259734
>logistic 0.685 (0.101)
>cart 0.746 (0.201)
>svm 0.804 (0.149)
>hard_voting 0.804 (0.149)


##AEDA

In [49]:
run_with_adea(data, "Ensemble")

results: [0.6 0.7]
Mean results 0.6499999999999999
Accuracy : 1.0
Precision : 1.0
Recall : 1.0
f1 score : 1.0
AUC score : 1.0
Matrix               precision    recall  f1-score   support

           0       1.00      1.00      1.00         3
           1       1.00      1.00      1.00         7

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

matthews Correlation coeffiecient: 1.0
>logistic 0.685 (0.101)
>cart 0.735 (0.195)
>svm 0.786 (0.155)
>hard_voting 0.786 (0.155)


##AugSBERT with EDA
###DA = EDA
###similarity = ce

In [50]:
run_with_augsbert(df,data,"Ensemble")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:EDA
Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

results: [0.8 0.6]
Mean results 0.7
Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0
>logistic 0.593 (0.154)
>cart 0.509 (0.224)
>svm 0.694 (0.039)
>hard_voting 0.593 (0.154)


##AugSBERT with Backtranslation
###DA = backtranslation
###similarity = ce

In [51]:
run_with_augsbert(df,data,"Ensemble")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:backtranslation


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 512 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

results: [0.6 0.6]
Mean results 0.6
Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0
>logistic 0.667 (0.068)
>cart 0.565 (0.146)
>svm 0.694 (0.039)
>hard_voting 0.667 (0.068)


##AugSBERT with AEDA
###DA = AEDA
###similarity = ce

In [52]:
run_with_augsbert(df,data,"Ensemble")

/usr/local/lib/python3.7/dist-packages/transformers/models/marian/tokenization_marian.py:198: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Select the method of augmentation:AEDA
Select the method of similarity:ce


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15 [00:00<?, ?it/s]

results: [0.4 0.6]
Mean results 0.5
Accuracy : 0.6
Precision : 0.6
Recall : 1.0
f1 score : 0.75
AUC score : 0.5
Matrix               precision    recall  f1-score   support

           0       1.00      0.00      0.00         2
           1       0.60      1.00      0.75         3

    accuracy                           0.60         5
   macro avg       0.80      0.50      0.37         5
weighted avg       0.76      0.60      0.45         5

matthews Correlation coeffiecient: 0.0
>logistic 0.667 (0.068)
>cart 0.602 (0.199)
>svm 0.694 (0.039)
>hard_voting 0.667 (0.068)
